In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, FloatType, BooleanType, ArrayType, IntegerType

In [2]:
#Upload do Ficheiro do tipo CSV para o HDFS
from os import PathLike
from hdfs import InsecureClient

client= InsecureClient("http://hdfs-nn:9870/",user="anonymous")

from_path="./PremierLeague.csv"
to_path="/demo/bronze/PremierLeague.csv"
client.delete(to_path)

client.upload(to_path,from_path)

'/demo/bronze/PremierLeague.csv'

In [3]:
#Configuração do Spark

warehouse_location = 'hdfs://hdfs-nn:9000/demo/silver'

builder = SparkSession     .builder     .appName("Python Spark SQL Hive integration example")     .config("spark.sql.warehouse.dir", warehouse_location)     .config("hive.metastore.uris", "thrift://hive-metastore:9083")     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")     .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")     .enableHiveSupport() 
spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2233404e-b57d-45f8-af23-4c60018a6b42;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 534ms :: artifacts dl 23ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [4]:
#Ver DataBases existentes no HDFS

spark.sql(
"""
SHOW DATABASES

"""
).show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+--------------+
|     namespace|
+--------------+
|       default|
|          demo|
|  league_stats|
|premier_league|
+--------------+



In [5]:
#Fazer o Drop

spark.sql(
    """
    DROP DATABASE IF EXISTS premier_league CASCADE
    """
)

DataFrame[]

In [6]:
#Ver DataBases existentes no HDFS

spark.sql(
"""
SHOW DATABASES

"""
).show()

+------------+
|   namespace|
+------------+
|     default|
|        demo|
|league_stats|
+------------+



In [7]:
#Criação premier_league.db

spark.sql(
"""
CREATE DATABASE premier_league LOCATION 'hdfs://hdfs-nn:9000/demo/silver/premier_league.db/'
"""
)

DataFrame[]

In [8]:
spark.sql(
"""
SHOW TABLES FROM premier_league

"""
).show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [9]:
spark.sql(
"""
DROP TABLE IF EXISTS premier_league.deltalake_table
"""
)

DataFrame[]

In [10]:
#Ver DataLake Table

spark.sql(
"""
SELECT *
FROM premier_league.deltalake_table
"""
).show()

AnalysisException: Table or view not found: premier_league.deltalake_table; line 3 pos 5;
'Project [*]
+- 'UnresolvedRelation [premier_league, deltalake_table], [], false


In [11]:
#Criação da DeltaLake Table

spark.sql(
"""
CREATE EXTERNAL TABLE premier_league.deltalake_table(
    Div CHAR(10),
    HomeTeam CHAR(100),
    AwayTeam CHAR(100),
    Referee CHAR(100),
    FTHG INT,
    FTAG INT,
    HF INT,
    AF INT,
    HY INT,
    AY INT,
    HR INT,
    AR INT
    )
    USING DELTA
    PARTITIONED BY (
         Date CHAR(50)
    )
    LOCATION 'hdfs://hdfs-nn:9000/demo/silver/premier_league.db/deltalake_table/'
"""
)

21/12/08 18:22:31 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `premier_league`.`deltalake_table` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
21/12/08 18:22:31 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [12]:
#Ver DeltaLake Table

spark.sql(
"""
SELECT *
FROM premier_league.deltalake_table
"""
).show()

+---+--------+--------+-------+----+----+---+---+---+---+---+---+----+
|Div|HomeTeam|AwayTeam|Referee|FTHG|FTAG| HF| AF| HY| AY| HR| AR|Date|
+---+--------+--------+-------+----+----+---+---+---+---+---+---+----+
+---+--------+--------+-------+----+----+---+---+---+---+---+---+----+



In [13]:
#Ler Ficheiro CSV No HDFS e Colocar no DataFrame

hdfs_path = "hdfs://hdfs-nn:9000/demo/bronze/PremierLeague.csv"

#Definir o Schema no DataFrame
customSchema = StructType([
    StructField("Div", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("HomeTeam", StringType(), True),        
    StructField("AwayTeam", StringType(), True),
    StructField("FTHG", IntegerType(), True),
    StructField("FTAG", IntegerType(), True),
    StructField("FTR", StringType(), True),
    StructField("HTHG", IntegerType(), True),
    StructField("HTAG", IntegerType(), True),
    StructField("HTR", StringType(), True),
    StructField("Referee", StringType(),True),
    StructField("HS", IntegerType(), True),
    StructField("AS", IntegerType(), True),
    StructField("HST", IntegerType(), True),
    StructField("AST", IntegerType(), True),
    StructField("HF", IntegerType(), True),
    StructField("AF", IntegerType(), True),
    StructField("HC", IntegerType(), True),
    StructField("AC", IntegerType(), True),
    StructField("HY", IntegerType(), True),
    StructField("AY", IntegerType(), True),
    StructField("HR", IntegerType(), True),
    StructField("AR", IntegerType(), True),
    StructField("B365H", FloatType(), True),
    StructField("B365D", FloatType(), True),
    StructField("B365A", FloatType(), True),
    StructField("BWH", FloatType(), True),
    StructField("BWD", FloatType(), True),
    StructField("BWA", FloatType(), True),
    StructField("IWH", FloatType(), True),
    StructField("IWD", FloatType(), True),
    StructField("IWA", FloatType(), True),
    StructField("PSH", FloatType(), True),
    StructField("PSD", FloatType(), True),
    StructField("PSA", FloatType(), True),
    StructField("WHH", FloatType(), True),
    StructField("WHD", FloatType(), True),
    StructField("WHA", FloatType(), True),
    StructField("VCH", FloatType(), True),
    StructField("VCD", FloatType(), True),
    StructField("VCA", FloatType(), True),
    StructField("Bb1X2", IntegerType(), True),
    StructField("BbMxH", FloatType(), True),
    StructField("BbAvH", FloatType(), True),
    StructField("BbMxD", FloatType(), True),
    StructField("BbAvD", FloatType(), True),
    StructField("BbMxA", FloatType(), True),
    StructField("BbAvA", FloatType(), True),
    StructField("BbOU", IntegerType(), True),
    StructField("BbMx>2.5", FloatType(), True),
    StructField("BbAv>2.5", FloatType(), True),
    StructField("BbMx<2.5", FloatType(), True),
    StructField("BbAv<2.5", FloatType(), True),
    StructField("BbAH", IntegerType(), True),
    StructField("BbAHh", FloatType(), True),
    StructField("BbMxAHH", FloatType(), True),
    StructField("BbAvAHH", FloatType(), True),
    StructField("BbMxAHA", FloatType(), True),
    StructField("BbAvAHA", FloatType(), True),
    StructField("PSCH", FloatType(), True),
    StructField("PSCD", FloatType(), True),
    StructField("PSCA", FloatType(), True),
])

sales_df = spark             .read            .option("delimiter",",")            .option("header","true")            .schema(customSchema)             .csv(hdfs_path)
sales_df.show()
sales_df.printSchema()

21/12/08 18:22:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----------+-----------+----------+----+----+---+----+----+---+-------------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+----+--------+--------+--------+--------+----+-----+-------+-------+-------+-------+-----+----+----+
|Div|      Date|   HomeTeam|  AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|      Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA| PSH| PSD| PSA| WHH| WHD| WHA| VCH| VCD| VCA|Bb1X2|BbMxH|BbAvH|BbMxD|BbAvD|BbMxA|BbAvA|BbOU|BbMx>2.5|BbAv>2.5|BbMx<2.5|BbAv<2.5|BbAH|BbAHh|BbMxAHH|BbAvAHH|BbMxAHA|BbAvAHA| PSCH|PSCD|PSCA|
+---+----------+-----------+----------+----+----+---+----+----+---+-------------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+----+-

21/12/08 18:22:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 71, schema size: 62
CSV file: hdfs://hdfs-nn:9000/demo/bronze/PremierLeague.csv


In [14]:
#Escrever a tabela criada anteriormente no DELTALAKE

sales_df     .select("Div", "HomeTeam", "AwayTeam", "Referee", "FTHG", "FTAG", "HF", "AF", "HY", "AY", "HR", "AR", "Date")     .write     .mode("overwrite")     .partitionBy("Date")     .format("delta")     .save("hdfs://hdfs-nn:9000/demo/silver/premier_league.db/deltalake_table/")

In [ ]:
#Demonstração dos dados da Tabela

spark.sql(
    """
    SELECT * FROM premier_league.deltalake_table
    """
).show()

In [ ]:
spark.sql(
"""
SELECT HomeTeam,AwayTeam,HF,AF
FROM premier_league.deltalake_table
"""
).show()

In [ ]:
#Fazer Verificações
spark.sql(
    """
    SELECT COUNT(*) FROM premier_league.deltalake_table
    
    """
).show()


In [ ]:
#Alteração da Coluna Div

spark.sql(
"""
UPDATE premier_league.deltalake_table
SET Div = REPLACE(Div,'E0','Premier League')
"""
)

In [ ]:
spark.sql(
"""
SELECT Date FROM premier_league.deltalake_table

"""
).show()

In [ ]:
spark.sql(
"""
SELECT *
FROM premier_league.deltalake_table
"""
).show()

In [ ]:
#Tratamento de Dados

sales_df = spark.read.format("delta").load("hdfs://hdfs-nn:9000/demo/silver/premier_league.db/deltalake_table/")

In [ ]:
#Alteração do nome de colunas

sales_df    .withColumnRenamed("Div", "Division")    .withColumnRenamed("FTHG", "HomeT_Goals")    .withColumnRenamed("FTAG", "AwayT_Goals")    .withColumnRenamed("HF", "HomeT_Fouls")    .withColumnRenamed("AF", "AwayT_Fouls")    .withColumnRenamed("HY", "HomeT_Yellow")    .withColumnRenamed("AY", "AwayT_Yellow")     .withColumnRenamed("HR", "HomeT_Red")    .withColumnRenamed("AR", "AwayT_Red")

In [ ]:
#Verificar a Existencia de Valores Duplicados no DeltaLake

import pyspark.sql.functions as f
sales_df.join(
    sales_df.groupBy(sales_df.columns).agg((f.count("*")>1).cast("int").alias("Duplicate_indicator")),
    on=sales_df.columns,
    how="inner"
).show()

In [ ]:
#Verificação de valores Distintos em HomeTeam,AwayTeam e Referee

sales_df.select("HomeTeam").distinct().collect()

In [ ]:
sales_df.select("AwayTeam").distinct().collect()

In [ ]:
sales_df.select("Referee").distinct().collect()

In [ ]:
#Verificação de Valores NULOS nas Tabelas

sales_df.filter(sales_df.HomeTeam.isNull()).count()

In [ ]:
sales_df.filter(sales_df.AwayTeam.isNull()).count()

In [ ]:
sales_df.filter(sales_df.HF.isNull()).count()

In [ ]:
sales_df.filter(sales_df.AF.isNull()).count()

In [ ]:
sales_df.filter(sales_df.Div.isNull()).count()

In [ ]:
sales_df.filter(sales_df.FTHG.isNull()).count()

In [ ]:
sales_df.filter(sales_df.FTAG.isNull()).count()

In [ ]:
sales_df.filter(sales_df.HY.isNull()).count()

In [ ]:
sales_df.filter(sales_df.AY.isNull()).count()

In [ ]:
sales_df.filter(sales_df.HR.isNull()).count()

In [ ]:
sales_df.filter(sales_df.AR.isNull()).count()

In [ ]:
spark.stop()